# Callbacks

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
os.listdir('dataset/cifar/')

['.DS_Store', 'test', 'labels.txt', 'train']

In [6]:
train_paths = glob('dataset/cifar/train/*.png')[:1000]  # 보기용으로 데이터 갯수를 1000개로 제한
test_paths = glob('dataset/cifar/test/*.png')[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

In [7]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [8]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [9]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [10]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [11]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [12]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [14]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Callbacks

In [15]:
logdir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))
logdir

'logs/20210218-143604'

In [19]:
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=logdir,
                                           write_graph=True,
                                           write_images=True,
                                           histogram_freq=1)

https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/r2/image_summaries.ipynb#scrollTo=IJNpyVyxbVtT

In [21]:
%tensorboard --logdir logs --port 8008

Reusing TensorBoard on port 8008 (pid 96389), started 0:01:05 ago. (Use '!kill 96389' to kill it.)

## Training

http://localhost:6006

In [20]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard]
)

/Users/lecturer/opt/anaconda3/envs/cvtf2/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
31/31 [==============================] - 5s 99ms/step - loss: 2.3332 - accuracy: 0.1056 - val_loss: 2.2996 - val_accuracy: 0.1129
Epoch 2/10
31/31 [==============================] - 3s 81ms/step - loss: 2.2952 - accuracy: 0.0909 - val_loss: 2.2433 - val_accuracy: 0.2026
Epoch 3/10
31/31 [==============================] - 2s 78ms/step - loss: 2.2348 - accuracy: 0.1381 - val_loss: 2.1206 - val_accuracy: 0.2006
Epoch 4/10
31/31 [==============================] - 3s 82ms/step - loss: 2.1574 - accuracy: 0.1794 - val_loss: 2.0903 - val_accuracy: 0.2510
Epoch 5/10
31/31 [==============================] - 3s 83ms/step - loss: 2.1281 - accuracy: 0.1872 - val_loss: 1.9970 - val_accuracy: 0.2611
Epoch 6/10
31/31 [==============================] - 3s 84ms/step - loss: 2.0292 - accuracy: 0.2326 - val_loss: 1.9520 - val_accuracy: 0.2520
Epoch 7/10
31/31 [==============================] - 3s 84ms/step - loss: 2.0206 - accuracy: 0.2408 - val_loss: 1.9581 - val_accuracy: 0.2651
Epoch 8/10
31

## Learning Rate Scheduler

In [24]:
import math
def scheduler(epoch):
    if epoch < 10:
        return 0.001
    else:
        return 0.001 * math.exp(0.1 * (10 - epoch))
learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [25]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[learning_rate_scheduler]
)

/Users/lecturer/opt/anaconda3/envs/cvtf2/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
31/31 [==============================] - 3s 89ms/step - loss: 1.8252 - accuracy: 0.2965 - val_loss: 1.7945 - val_accuracy: 0.3095
Epoch 2/10
31/31 [==============================] - 3s 88ms/step - loss: 1.8147 - accuracy: 0.2903 - val_loss: 1.8329 - val_accuracy: 0.3175
Epoch 3/10
31/31 [==============================] - 2s 81ms/step - loss: 1.7632 - accuracy: 0.3285 - val_loss: 1.8581 - val_accuracy: 0.3075
Epoch 4/10
31/31 [==============================] - 3s 84ms/step - loss: 1.7877 - accuracy: 0.3140 - val_loss: 1.7358 - val_accuracy: 0.3226
Epoch 5/10
31/31 [==============================] - 3s 83ms/step - loss: 1.6810 - accuracy: 0.3574 - val_loss: 1.7337 - val_accuracy: 0.3286
Epoch 6/10
31/31 [==============================] - 3s 86ms/step - loss: 1.6782 - accuracy: 0.3440 - val_loss: 1.7066 - val_accuracy: 0.3629
Epoch 7/10
31/31 [==============================] - 3s 87ms/step - loss: 1.6594 - accuracy: 0.3750 - val_loss: 1.7427 - val_accuracy: 0.3357
Epoch 8/10
31

https://www.tensorflow.org/tensorboard/r2/image_summaries#setup

### LambdaCallback

In [26]:
import sklearn.metrics
import itertools
import io

file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

test_images, test_labels = next(iter(test_dataset))  # Confusion Matrix 그릴 때 필요한 Test Image

def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [27]:
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [28]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard, cm_callback]
)

/Users/lecturer/opt/anaconda3/envs/cvtf2/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
31/31 [==============================] - 3s 89ms/step - loss: 1.5430 - accuracy: 0.3988 - val_loss: 1.7203 - val_accuracy: 0.3599
Epoch 2/10
31/31 [==============================] - 3s 91ms/step - loss: 1.5912 - accuracy: 0.3874 - val_loss: 1.7785 - val_accuracy: 0.3438
Epoch 3/10
31/31 [==============================] - 3s 91ms/step - loss: 1.5452 - accuracy: 0.4163 - val_loss: 1.6629 - val_accuracy: 0.3438
Epoch 4/10
31/31 [==============================] - 3s 94ms/step - loss: 1.5090 - accuracy: 0.4395 - val_loss: 1.6485 - val_accuracy: 0.3942
Epoch 5/10
31/31 [==============================] - 3s 94ms/step - loss: 1.4493 - accuracy: 0.4544 - val_loss: 1.6330 - val_accuracy: 0.3982
Epoch 6/10
31/31 [==============================] - 3s 90ms/step - loss: 1.4554 - accuracy: 0.4504 - val_loss: 1.6612 - val_accuracy: 0.4083
Epoch 7/10
31/31 [==============================] - 3s 98ms/step - loss: 1.4508 - accuracy: 0.4638 - val_loss: 1.6674 - val_accuracy: 0.3800
Epoch 8/10
31

## Expert

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)  # Loss 계산
    gradients = tape.gradient(loss, model.trainable_variables)  # 모델의 trainable_variable을 하여금 loss를 통해 기울기를 얻음
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # 구한 최적화된 값을 variable에 적용

    train_loss(loss)
    train_accuracy(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [ ]:
logdir = os.path.join('logs',  datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
file_writer = tf.summary.create_file_writer(logdir)

In [ ]:
for epoch in range(num_epochs):
    for step, (images, labels) in enumerate(train_dataset):            
        train_step(images, labels)

    with file_writer.as_default():
        tf.summary.image('input_image', images, step=step)

    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))